In [224]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [225]:
# 폴더 이동
%cd /content/drive/MyDrive/Colab Notebooks/comment

/content/drive/MyDrive/Colab Notebooks/KoGPT2-DiaryCommentAI/comment


In [ ]:
!pip install -r requirements.txt
!pip install torch==1.8.0+cu111 torchvision==0.9.0+cu111 torchaudio==0.8.0 -f https://download.pytorch.org/whl/torch_stable.html

일기 문단 전체를 diary에 저장합니다.

In [226]:
diary = '어릴 적부터 태극기를 달아야 하는 날 3.1절, 제헌절, 광복절, 개천절, 한글날, 현충일 에는 아침에 일어나 국기부터 달았던 것 같다. 철저히 체크 하시는 우리 아버지. 당연스럽게도 결혼 후에 태극기를 주문했다. 나도 내 자녀들에게도 마음을 잘 심어주고 싶다는 생각이었다. 해야 하는 것이고 의미 있고 좋은 실천이다. 오늘은 조의를 표하는 현충일 이기에 조기를 달아야 하는데. 조기는 깃면의 넓이(세로) 만큼 내려 달아야 한다. 완전한 조기를 달 수 없으면 최대한 닿지 않게 내려서 달면 된다고 한다. 우리 집은 밑 부분이 최대한 잘 고정될 만큼의 안전한 선에서 달았다. 좋은 실천을 한 것 같아 마음이 뿌듯했다.'

문장의 위치에 따라 중요도를 다르게 설정하였습니다.
일기이므로, 마지막 문장이 다른 문장보다 10배 중요하다고 가정하였습니다.
numpy.ndarray 형태로 bias 를 만든 다음, 이를 summarize 함수의 bias 에 입력하면 가장 먼저 맨 마지막 문장이 중요한 문장으로 선택됩니다. 
다른 문장들 중에서도 맨 마지막 문장과 비슷할수록 상대적인 중요도가 더 커집니다.

In [227]:
import numpy as np

sents_list = diary.split('.')
sents =[]
for sent in sents_list:
    if sent.strip() is not '':
      sents.append(sent.strip())
      
bias = np.ones(len(sents))
bias[-1] = 10
keysents_list = summarizer.summarize(sents, topk=3, bias=bias)
keysents = []
for _, _, sent in keysents_list:
    keysents.append(sent+'.')

print(keysents)

['좋은 실천을 한 것 같아 마음이 뿌듯했다.', '1절, 제헌절, 광복절, 개천절, 한글날, 현충일 에는 아침에 일어나 국기부터 달았던 것 같다.', '해야 하는 것이고 의미 있고 좋은 실천이다.']


In [228]:
import torch
from pytorch_lightning.core.lightning import LightningModule
from transformers import PreTrainedTokenizerFast, GPT2LMHeadModel

# KoGPT2 모델에서 사용하는 특수 토큰
U_TKN = '<usr>' # 일기 핵심 문장의 시작을 나타내는 특수 토큰
S_TKN = '<sys>' # 코멘트의 시작을 나타내는 특수 토큰
BOS = '</s>'    # 문장의 시작을 나타내는 특수 토큰
EOS = '</s>'    # 문장의 끝을 나타내는 특수 토큰
UNK = '<unk>'   # 어휘에 없는 토큰을 나타내는 특수 토큰
MASK = '<unused0>'  # 마스킹된 토큰을 나타내는 특수 토큰
SENT = '<unused1>'  # 일기 핵심 문장의 끝을 나타내는 특수 토큰
PAD = '<pad>'   # 토큰 배열을 동일한 크기로 만드는데 사용되는 특수 토큰

# KoGPT2에서 제공하는 토큰나이저 사용
TOKENIZER = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2",
            bos_token=BOS, eos_token=EOS, unk_token=UNK,
            pad_token=PAD, mask_token=MASK)
class KoGPT2Comment(LightningModule):
    def __init__(self, hparams, **kwargs):
        super(KoGPT2Comment, self).__init__()
        self.kogpt2 = GPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2') # pretrained KoGPT2 model
    def forward(self, inputs):
        # (batch, seq_len, hiddens)
        output = self.kogpt2(inputs, return_dict=True)
        return output.logits
    def comment(self, keysent):
        tok = TOKENIZER
        sent='0'
        sent_tokens = tok.tokenize(sent)
        with torch.no_grad():
                q = keysent
                a = ''
                while 1:
                    input_ids = torch.LongTensor(tok.encode(U_TKN + q + SENT + sent + S_TKN + a)).unsqueeze(dim=0)
                    pred = self(input_ids)
                    gen = tok.convert_ids_to_tokens(
                        torch.argmax(
                            pred,
                            dim=-1).squeeze().numpy().tolist())[-1]
                    if gen == EOS:
                        break
                    a += gen.replace('▁', ' ')
                return a.strip()

model = KoGPT2Comment.load_from_checkpoint('model_chp/model_-last.ckpt')
moon_comment = ''
max_comment = ''
comment = ''
for sent in keysents:
  comment = model.comment(sent)
  if(comment[-1] is not '?' and (not '.' or not '!')):
        comment = comment + '.'
  if(len(max_comment)<=len(comment)):
        max_comment = comment
  if(len(comment)>6 and '?' not in comment and '저' not in comment):
        moon_comment = comment
        break
if(moon_comment is ''):
  moon_comment = max_comment
print(moon_comment)


좋은 실천이네요.
